# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2016-07-20 to 2021-07-20
You have 500 stocks
['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL
date,,,,,,,,,,,,,,,,,,,,,
2021-07-14 00:00:00+00:00,148.63,20.62,210.16,149.150,117.36,113.56,312.30,117.17,314.390,608.83,...,51.260,371.44,23.580,34.70,110.7100,68.09,132.74,59.530,62.16,121.01
2021-07-15 00:00:00+00:00,148.96,20.46,209.25,148.480,117.18,113.16,310.19,116.66,314.940,606.17,...,50.290,371.96,23.440,34.63,108.2300,68.72,129.71,58.950,61.80,120.01
2021-07-16 00:00:00+00:00,148.93,19.79,205.85,146.390,117.50,112.88,317.98,117.51,311.910,606.10,...,49.010,373.66,23.500,34.02,104.7000,69.13,128.63,57.320,61.16,119.36
2021-07-19 00:00:00+00:00,147.58,18.97,205.67,142.450,115.42,112.60,314.62,118.79,309.180,602.05,...,47.990,371.00,22.720,32.95,104.7400,68.27,129.33,55.350,60.69,117.23
2021-07-20 00:00:00+00:00,149.59,20.20,211.18,146.765,115.94,116.67,323.96,118.82,313.815,610.37,...,48.665,375.57,23.105,33.52,106.3567,68.66,130.92,56.165,62.55,121.55


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.FactorMomentum(price_histories, 252).demean().rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().smoothed().rank().zscore().for_al(),
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.MarketDispersion(price_histories, 20).for_al(),
    alpha_factors.MarketDispersion(price_histories, 120).for_al(),
    alpha_factors.MarketVolatility(price_histories, 20).for_al(),
    alpha_factors.MarketVolatility(price_histories, 120).for_al()
], axis=1)
alpha_factors.FactorDateParts(all_factors)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAPL'].tail()

,,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day,average_dollar_volume_20_day,average_dollar_volume_120_day,market_dispersion20_day,market_dispersion120_day,market_volatility20_day,market_volatility120_day,is_January,is_December,weekday,quarter,year,month_start,month_end,quarter_start,quarter_end
date,ticker,,,,,,,,,,,,,,,,,,,,
2021-07-14 00:00:00+00:00,AAPL,0.481511,-1.707806,0.176670,0.980343,-0.599291,1.714734,1.721662,0.012697,0.016262,0.109875,0.104522,0,0,2,3,2021,0,0,0,0
2021-07-15 00:00:00+00:00,AAPL,0.578506,-1.707806,0.737855,0.987271,-0.640860,1.714734,1.721662,0.012811,0.016256,0.106578,0.104343,0,0,3,3,2021,0,0,0,0
2021-07-16 00:00:00+00:00,AAPL,0.640860,-1.707806,0.945702,0.897204,-0.717070,1.714734,1.721662,0.012416,0.016236,0.105082,0.105340,0,0,4,3,2021,0,0,0,0
2021-07-19 00:00:00+00:00,AAPL,0.627004,-1.700877,1.132764,0.571578,-0.786353,1.714734,1.721662,0.012572,0.016181,0.106672,0.107164,0,0,0,3,2021,0,0,0,0
2021-07-20 00:00:00+00:00,AAPL,0.474583,-1.714734,0.931845,0.370660,-0.911061,1.714734,1.721662,0.015219,0.016490,0.105628,0.106115,0,0,1,3,2021,0,0,0,0


In [6]:
all_factors.describe()

,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day,average_dollar_volume_20_day,average_dollar_volume_120_day,market_dispersion20_day,market_dispersion120_day,market_volatility20_day,market_volatility120_day,is_January,is_December,weekday,quarter,year,month_start,month_end,quarter_start,quarter_end
count,5.035000e+05,5.035000e+05,5.035000e+05,5.035000e+05,5.035000e+05,5.035000e+05,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000,503500.000000
mean,-4.798104e-19,3.422177e-19,-2.169731e-19,-7.408837e-20,-1.622888e-19,-3.175216e-19,0.000000,0.016609,0.016470,0.131433,0.129637,0.081430,0.081430,2.020854,2.514399,2019.047666,0.041708,0.045680,0.011917,0.014896
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001,0.005798,0.003907,0.126631,0.079723,0.273495,0.273495,1.401718,1.113938,1.222803,0.199921,0.208791,0.108511,0.121136
min,-1.728597e+00,-1.728594e+00,-1.728590e+00,-1.728590e+00,-1.728590e+00,-1.728590e+00,-1.728590,0.010485,0.012020,0.026211,0.044904,0.000000,0.000000,0.000000,1.000000,2017.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.660289e-01,-8.642964e-01,-8.660272e-01,-8.642964e-01,-8.642964e-01,-8.642964e-01,-0.864296,0.013266,0.014123,0.066871,0.086200,0.000000,0.000000,1.000000,2.000000,2018.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.015158,0.014706,0.099438,0.106728,0.000000,0.000000,2.000000,3.000000,2019.000000,0.000000,0.000000,0.000000,0.000000
75%,8.642976e-01,8.625648e-01,8.660272e-01,8.642964e-01,8.642964e-01,8.642964e-01,0.864296,0.017570,0.018372,0.151125,0.137489,0.000000,0.000000,3.000000,4.000000,2020.000000,0.000000,0.000000,0.000000,0.000000
max,1.728597e+00,1.728594e+00,1.728590e+00,1.728594e+00,1.728594e+00,1.728594e+00,1.728594,0.050857,0.028105,0.909613,0.375388,1.000000,1.000000,4.000000,4.000000,2021.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
all_factors.head()

momentum_252_day_logret  \
date                      ticker                            
2017-07-20 00:00:00+00:00 A                      0.710144   
                          AAL                    1.229761   
                          AAP                   -1.687024   
                          AAPL                   1.396039   
                          ABBV                   0.072746   

                                  mean_reversion_5_day_logret_smoothed  \
date                      ticker                                         
2017-07-20 00:00:00+00:00 A                                  -0.779426   
                          AAL                                -1.638527   
                          AAP                                 1.728594   
                          AAPL                               -1.139694   
                          ABBV                               -0.495369   

                                  overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2017-07-20 00:00:00+00:00 A                                 1.292112   
                          AAL                               0.675501   
                          AAP                              -1.132763   
                          AAPL                             -1.451461   
                          ABBV                             -0.917989   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2017-07-20 00:00:00+00:00 A                         -0.239024   
                          AAL                        0.807139   
                          AAP                        1.520747   
                          AAPL                       1.264402   
                          ABBV                       0.647790   

                                  annualzed_volatility_120_day  \
date                      ticker                                 
2017-07-20 00:00:00+00:00 A                           0.287522   
                          AAL                         0.419158   
                          AAP                         1.603885   
                          AAPL                       -0.620077   
                          ABBV                        0.370660   

                                  average_dollar_volume_20_day  \
date                      ticker                                 
2017-07-20 00:00:00+00:00 A                          -0.412230   
                          AAL                         1.035771   
                          AAP                         0.793282   
                          AAPL                        1.728594   
                          ABBV                        1.195120   

                                  average_dollar_volume_120_day  \
date                      ticker                                  
2017-07-20 00:00:00+00:00 A                           -0.550794   
                          AAL                          1.021914   
                          AAP                          0.495369   
                          AAPL                         1.728594   
                          ABBV                         1.250546   

                                  market_dispersion20_day  \
date                      ticker                            
2017-07-20 00:00:00+00:00 A                      0.012224   
                          AAL                    0.012224   
                          AAP                    0.012224   
                          AAPL                   0.012224   
                          ABBV                   0.012224   

                                  market_dispersion120_day  \
date                      ticker                             
2017-07-20 00:00:00+00:00 A                       0.012114   
                          AAL                     0.012114   
                          AAP                     0.01

In [8]:
all_factors.tail()

momentum_252_day_logret  \
date                      ticker                            
2021-07-20 00:00:00+00:00 XEL                   -1.368323   
                          XLNX                  -0.516152   
                          XOM                   -0.294449   
                          XRAY                   0.301377   
                          XYL                    1.021912   

                                  mean_reversion_5_day_logret_smoothed  \
date                      ticker                                         
2021-07-20 00:00:00+00:00 XEL                                -0.162813   
                          XLNX                                0.162813   
                          XOM                                 1.493031   
                          XRAY                                1.368323   
                          XYL                                -1.014984   

                                  overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2021-07-20 00:00:00+00:00 XEL                               0.890276   
                          XLNX                              0.488439   
                          XOM                              -1.513815   
                          XRAY                             -1.077338   
                          XYL                               0.301377   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2021-07-20 00:00:00+00:00 XEL                        0.647788   
                          XLNX                       1.222830   
                          XOM                        1.596954   
                          XRAY                      -1.652380   
                          XYL                       -1.271328   

                                  annualzed_volatility_120_day  \
date                      ticker                                 
2021-07-20 00:00:00+00:00 XEL                        -0.502296   
                          XLNX                        1.299041   
                          XOM                         1.153548   
                          XRAY                        0.176670   
                          XYL                        -1.680093   

                                  average_dollar_volume_20_day  \
date                      ticker                                 
2021-07-20 00:00:00+00:00 XEL                        -0.294449   
                          XLNX                        0.460726   
                          XOM                         1.583098   
                          XRAY                       -1.423749   
                          XYL                        -0.883348   

                                  average_dollar_volume_120_day  \
date                      ticker                                  
2021-07-20 00:00:00+00:00 XEL                         -0.155885   
                          XLNX                         0.495368   
                          XOM                          1.603882   
                          XRAY                        -1.292112   
                          XYL                         -1.271328   

                                  market_dispersion20_day  \
date                      ticker                            
2021-07-20 00:00:00+00:00 XEL                    0.015219   
                          XLNX                   0.015219   
                          XOM                    0.015219   
                          XRAY                   0.015219   
                          XYL                    0.015219   

                                  market_dispersion120_day  \
date                      ticker                             
2021-07-20 00:00:00+00:00 XEL                      0.01649   
                          XLNX                     0.01649   
                          XOM                      0.0